# Exemplo de tratamento de dados: Steam

In [1]:
import pandas as pd

dados = pd.read_csv('STC_set_data.csv')
dados.head()

,Game,# Owned,# Unique,# Cards,Badge Lvl,Set Price,Price Diff,Card Avg,Booster Avg,Booster %,Emote Avg,BG Avg,Avg Qty,Discount,Added,AppId
0,City Game Studio,0,0,12,0,1.25,1.25,0.10,NaN,0.0,0.00,0.00,17,0.52,2019-10-24,726840
1,Transport Services,0,0,5,0,1.85,1.85,0.37,NaN,0.0,0.00,0.00,4,0.97,2019-10-23,993690
2,Deadly Days,0,0,10,0,0.78,0.78,0.08,NaN,0.0,0.00,0.00,26,0.35,2019-10-22,740080
3,Moons of Madness,0,0,6,0,0.75,0.75,0.12,0.17,-41.7,0.00,0.00,36,0.31,2019-10-22,1012840
4,NEKO-NIN exHeart 3,0,0,6,0,1.00,1.00,0.17,0.18,-5.9,1.34,5.56,45,0.44,2019-10-21,1129250


In [2]:
qnt_de_itens_inicial = dados.shape[0]
qnt_de_itens_inicial

9156

## Remover os dados inúteis para o tratamento (trabalharemos com cartas e boosters)

In [3]:
dados = dados.drop(["# Owned", "# Unique", "Badge Lvl", "Emote Avg", "BG Avg", "Added"], axis=1)
dados.head()

,Game,# Cards,Set Price,Price Diff,Card Avg,Booster Avg,Booster %,Avg Qty,Discount,AppId
0,City Game Studio,12,1.25,1.25,0.10,NaN,0.0,17,0.52,726840
1,Transport Services,5,1.85,1.85,0.37,NaN,0.0,4,0.97,993690
2,Deadly Days,10,0.78,0.78,0.08,NaN,0.0,26,0.35,740080
3,Moons of Madness,6,0.75,0.75,0.12,0.17,-41.7,36,0.31,1012840
4,NEKO-NIN exHeart 3,6,1.00,1.00,0.17,0.18,-5.9,45,0.44,1129250


## Verificar os tipos presentes nas colunas estão aceitáveis:

In [4]:
formato_dos_dados = dados.dtypes
formato_dos_dados

Game            object
# Cards          int64
Set Price       object
Price Diff      object
Card Avg       float64
Booster Avg    float64
Booster %       object
Avg Qty          int64
Discount       float64
AppId            int64
dtype: object

#### É necessário fazer as seguintes conversões de tipos para trabalhar:
- 1. 'Set Price', 'Price Diff' e 'Booster %' devem ser do tipo float. Para Evitar erro de conversão é necessário substituir o caractere ',' por '' (espaço vazio)
- 2. 'AppId' por ser um código de identificação deve ser do tipo string para evitar problemas

In [5]:
def remove_virgula_perdida(sentenca):
    sentenca = sentenca.replace(',','')
    return sentenca

In [6]:
para_float = ['Set Price', 'Price Diff', 'Booster %']

for coluna in dados[para_float]:
    dados[coluna] = dados[coluna].apply(remove_virgula_perdida)
    
dados[para_float] = dados[para_float].astype('float64')
dados['AppId'] = dados['AppId'].astype('object')

In [7]:
dados.dtypes

Game            object
# Cards          int64
Set Price      float64
Price Diff     float64
Card Avg       float64
Booster Avg    float64
Booster %      float64
Avg Qty          int64
Discount       float64
AppId           object
dtype: object

In [8]:
dados.head()

,Game,# Cards,Set Price,Price Diff,Card Avg,Booster Avg,Booster %,Avg Qty,Discount,AppId
0,City Game Studio,12,1.25,1.25,0.10,NaN,0.0,17,0.52,726840
1,Transport Services,5,1.85,1.85,0.37,NaN,0.0,4,0.97,993690
2,Deadly Days,10,0.78,0.78,0.08,NaN,0.0,26,0.35,740080
3,Moons of Madness,6,0.75,0.75,0.12,0.17,-41.7,36,0.31,1012840
4,NEKO-NIN exHeart 3,6,1.00,1.00,0.17,0.18,-5.9,45,0.44,1129250


## É possível observar na tabela que alguns valores de 'Booster Avg' não estão presentes o que afeta diretamente a nossa análise. Portanto, faremos uma separação destes dados NaN para comparar

In [9]:
dados['Booster Avg'].value_counts(normalize = True, dropna = False).head(10)

0.03    0.150939
0.04    0.125382
NaN     0.090105
0.05    0.086501
0.06    0.063893
0.08    0.042267
0.09    0.038008
0.02    0.033967
0.10    0.032438
0.07    0.032219
Name: Booster Avg, dtype: float64

9% não possuem dados de entrada de média de booster, isso acontece por eles estarem raros, não possuindo ofertas no mercado

In [10]:
dados_nulos = dados[dados['Booster Avg'].isnull()]
dados_reais = dados[~dados['Booster Avg'].isnull()]

# Vamos começar analizando os dados com entrada nula de Boosters:

In [11]:
round(dados_nulos.describe(),2)

,# Cards,Set Price,Price Diff,Card Avg,Booster Avg,Booster %,Avg Qty,Discount
count,825.00,825.00,825.00,825.00,0.0,825.0,825.00,825.00
mean,6.78,6.63,6.63,1.08,NaN,0.0,19.10,1.85
std,2.19,108.19,108.19,18.06,NaN,0.0,14.57,17.64
min,5.00,0.35,0.35,0.05,NaN,0.0,1.00,0.07
25%,5.00,0.78,0.78,0.12,NaN,0.0,11.00,0.37
50%,6.00,1.09,1.09,0.17,NaN,0.0,17.00,0.50
75%,8.00,1.60,1.60,0.26,NaN,0.0,24.00,0.74
max,15.00,3065.79,3065.79,510.97,NaN,0.0,186.00,444.32


Não há motivos para mantermos as colunas 'Booster Avg' e 'Booster %' nesta análise pois ambos são nulos

In [12]:
dados_nulos = dados_nulos.drop(["Booster Avg", "Booster %"], axis=1)

In [14]:
round(dados_nulos.describe(),2)

,# Cards,Set Price,Price Diff,Card Avg,Avg Qty,Discount
count,825.00,825.00,825.00,825.00,825.00,825.00
mean,6.78,6.63,6.63,1.08,19.10,1.85
std,2.19,108.19,108.19,18.06,14.57,17.64
min,5.00,0.35,0.35,0.05,1.00,0.07
25%,5.00,0.78,0.78,0.12,11.00,0.37
50%,6.00,1.09,1.09,0.17,17.00,0.50
75%,8.00,1.60,1.60,0.26,24.00,0.74
max,15.00,3065.79,3065.79,510.97,186.00,444.32


In [16]:
round(dados_nulos.corr(),3)

,# Cards,Set Price,Price Diff,Card Avg,Avg Qty,Discount
# Cards,1.000,-0.010,-0.010,-0.017,0.023,-0.006
Set Price,-0.010,1.000,1.000,0.999,-0.060,0.943
Price Diff,-0.010,1.000,1.000,0.999,-0.060,0.943
Card Avg,-0.017,0.999,0.999,1.000,-0.059,0.945
Avg Qty,0.023,-0.060,-0.060,-0.059,1.000,-0.083
Discount,-0.006,0.943,0.943,0.945,-0.083,1.000
